# Notebook 1

In this notebook we accomplish the following items:
    * Ensure we are truly connected to cluster, and can download packages as required
    * Infer "State" when missing based on initial location of file and save to HDFS
    * Count number of missing data items for each row and write to HDFS (Necessary as time consuming)

In [1]:
%%sh 
# Can see we're in right spot
pwd

/home/wce/clsadmin


## Installing Packages we might need

In [2]:
%%sh
pip install python-dotenv
pip install plotly
pip install pyshp
pip install Shapely

  Using cached https://files.pythonhosted.org/packages/8c/14/501508b016e7b1ad0eb91bba581e66ad9bfc7c66fcacbb580eaf9bc38458/python_dotenv-0.10.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/32/d6/82333db5a6f56f828d817d49ae6ea153125d70214a189686afe784e159ad/plotly-3.4.2-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/7d/e3/20f3d364d6c8e5d2353c72a67778eb189176f08e873c9900e10c0287b84b/requests-2.21.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/da/27/9a654d2b6cc1eaa517d1c5a4405166c7f6d72f04f6e7eea41855fe808a46/nbformat-4.4.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/f8/0e/2365ddc010afb3d79147f1dd544e5ee24bf4ece58ab99b16fbb465ce6dc0/pytz-2018.7-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/73/fb/00a976f728d0d1fecfe898238ce23f502a721c0ac0ecfedb80e0d88c64e9/six-1.12.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages

pillow 4.0.0 requires olefile, which is not installed.
tensorflow 1.7.0 has requirement numpy>=1.13.3, but you'll have numpy 1.11.3 which is incompatible.
pyrax 1.9.8 has requirement python-novaclient==2.27.0, but you'll have python-novaclient 10.2.0 which is incompatible.
tensorboard 1.7.0 has requirement futures>=3.1.1; python_version < "3", but you'll have futures 3.0.5 which is incompatible.
tensorboard 1.7.0 has requirement numpy>=1.12.0, but you'll have numpy 1.11.3 which is incompatible.
Target directory /home/wce/clsadmin/pipAnaconda2Packages/dotenv already exists. Specify --upgrade to force replacement.
Target directory /home/wce/clsadmin/pipAnaconda2Packages/python_dotenv-0.10.1.dist-info already exists. Specify --upgrade to force replacement.
Target directory /home/wce/clsadmin/pipAnaconda2Packages/bin already exists. Specify --upgrade to force replacement.
pillow 4.0.0 requires olefile, which is not installed.
tensorflow 1.7.0 has requirement numpy>=1.13.3, but you'll have 

In [ ]:
# Packages needed for notebook
import plotly
import pandas
import dotenv

import math
from pyspark.sql.functions import udf, struct
from pyspark.sql.types import *
from pyspark.sql.functions import lit
from functools import reduce  # For Python 3.x
from pyspark.sql import DataFrame

import shapefile
from shapely.geometry import Point # Point class
from shapely.geometry import shape # shape() is a function to convert geo objects through the interface

# Function to concat dataframes
def unionAll(*dfs):
    return reduce(DataFrame.unionAll, dfs)



In [18]:
# define state list
statesCap = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", 
             "FL", "GA", "HI", "ID", "IL", "IN", "IA", "KS", "KY", 
             "LA", "ME", "MD", "MA", "MI", "MN", "MS", "MO", "MT", 
             "NE", "NV", "NH", "NJ", "NM", "NY", "NC", "ND", "OH", 
             "OK", "OR", "PA", "RI", "SC", "SD", "TN", "TX", "UT", 
             "VT", "VA", "WA", "WV", "WI", "WY"]
states = [x.lower() for x in statesCap]

In [5]:
# def add_state_and_dedupe(states):
#     """
#     Adds known state to each set of files, dedupes, and unions into a single df
#     """
#     first_state = True
#     for state in states:
#         pathway = "/data/%s/*" %state
#         print(pathway)
#         df = spark.read.csv(pathway, header=True).persist()
#         df = df.withColumn('State',lit(state))
#         df = df.drop_duplicates().persist()
#         df = df.drop('REGION').persist()
#         if first_state:
#             full_df = df
#             first_state=False
#         else:
#             full_df = unionAll(full_df, df).persist()
#     # Our full dataset size (takes some time) 
#     full_df = full_df.drop_duplicates().persist()  
#     return full_df


# full_df = add_state_and_dedupe(states)

In [6]:
def add_state_and_dedupe(states):
    """
    Adds known state to each set of files, dedupes, and unions into a single df
    """
    first_state = True
    for state in states:
        pathway = "/data/%s/*" %state
        print(pathway)
        df = spark.read.csv(pathway, header=True).persist()
        df = df.withColumn('State',lit(state))
        df = df.drop_duplicates().persist()
        df = df.drop('REGION').persist()
        output_path = '/clean_states/%s' %state
        df.write.csv(output_path, header=True)
    return "Write Complete"


add_state_and_dedupe(states)

/data/al/*
/data/ak/*
/data/az/*
/data/ar/*
/data/ca/*
/data/co/*
/data/ct/*
/data/dc/*
/data/de/*
/data/fl/*
/data/ga/*
/data/hi/*
/data/id/*
/data/il/*
/data/in/*
/data/ia/*
/data/ks/*
/data/ky/*
/data/la/*
/data/me/*
/data/md/*
/data/ma/*
/data/mi/*
/data/mn/*
/data/ms/*
/data/mo/*
/data/mt/*
/data/ne/*
/data/nv/*
/data/nh/*
/data/nj/*
/data/nm/*
/data/ny/*
/data/nc/*
/data/nd/*
/data/oh/*
/data/ok/*
/data/or/*
/data/pa/*
/data/ri/*
/data/sc/*
/data/sd/*
/data/tn/*
/data/tx/*
/data/ut/*
/data/vt/*
/data/va/*
/data/wa/*
/data/wv/*
/data/wi/*
/data/wy/*


'Write Complete'

In [23]:
from pyspark.sql.functions import udf, struct
from pyspark.sql.types import IntegerType

def count_missing(states):
    """
    Adds known state to each set of files, dedupes, and unions into a single df
    """
    for state in states:
        pathway = "/clean_states/%s/*" %state
        print(pathway)
        df = spark.read.csv(pathway, header=True).persist()
        count_empty_columns = udf(lambda row: len([x for x in row if x == None]), IntegerType())
        new_df = df.withColumn("null_count", count_empty_columns(struct([df[x] for x in df.columns])))
        output_path = '/states_missingness/%s' %state
        new_df.write.csv(output_path, header=True)
    return "Write Complete"


count_missing(states)

/clean_states/al/*
/clean_states/ak/*
/clean_states/az/*
/clean_states/ar/*
/clean_states/ca/*
/clean_states/co/*
/clean_states/ct/*
/clean_states/dc/*
/clean_states/de/*
/clean_states/fl/*
/clean_states/ga/*
/clean_states/hi/*
/clean_states/id/*
/clean_states/il/*
/clean_states/in/*
/clean_states/ia/*
/clean_states/ks/*
/clean_states/ky/*
/clean_states/la/*
/clean_states/me/*
/clean_states/md/*
/clean_states/ma/*
/clean_states/mi/*
/clean_states/mn/*
/clean_states/ms/*
/clean_states/mo/*
/clean_states/mt/*
/clean_states/ne/*
/clean_states/nv/*
/clean_states/nh/*
/clean_states/nj/*
/clean_states/nm/*
/clean_states/ny/*
/clean_states/nc/*
/clean_states/nd/*
/clean_states/oh/*
/clean_states/ok/*
/clean_states/or/*
/clean_states/pa/*
/clean_states/ri/*
/clean_states/sc/*
/clean_states/sd/*
/clean_states/tn/*
/clean_states/tx/*
/clean_states/ut/*
/clean_states/vt/*
/clean_states/va/*
/clean_states/wa/*
/clean_states/wv/*
/clean_states/wi/*
/clean_states/wy/*


'Write Complete'

Supplemental data cleaning was done directly through scripts as processes were too compute intense

# End Notebook